In [1]:
from gurobi_models import *
import time
import shutil
from datetime import datetime
from timeit import default_timer as timer
import argparse
import gurobipy as gp
from gurobipy import GRB

In [2]:
def run_from_config(config_file, save_variables=False, run_time = 600, seed = None, base_file_name = 'test'):
   '''runs the model from a config file'''
   test_instances = []
   env = gp.Env("gurobi.log")
   with open(config_file) as f:
      print('Opening config file', config_file)
      print('base_file_name', base_file_name)
      #Removes file extension from config file name
      conf_name = config_file.split('.')[0].split('/')[-1]
      print('conf_name', conf_name)
      xp_yaml = yaml.load(f, Loader=yaml.FullLoader)
      #configuring problem
      SEQUENCE_LENGTH = xp_yaml['sequence_length']
      NO_WORKERS = xp_yaml['max_workers']
      NO_STATIONS = xp_yaml['no_stations']
      WORKER_COST = xp_yaml['worker_cost']
      RECOURSE_COST = xp_yaml['recourse_cost']
      #configuring scenario tree generator
      tree_kwargs, scenario_generator = get_scenario_generator(xp_yaml, seed)
      #copying config file to results folder
      print('copying config file to results folder',  config_file, base_file_name +'/'+ conf_name + '_config.yaml')
      shutil.copyfile(config_file, base_file_name +'/'+ conf_name + '_config.yaml')
      for milp_model in xp_yaml['milp_models']:
        if milp_model == 'dynamic_problem_linear_labor_recourse':
            milp_model = dynamic_MMALBP_W
            file_name = base_file_name + '/dynamic_linear/'
            #if model_dependent directory does not exist, make it
            if not os.path.exists(file_name):
               os.makedirs(file_name)
            file_name = file_name + 'ld_'
        else:
            raise ValueError('milp_model not recognized')
         #Keeps track of time
        start_time = time.time()
        group_counter = 0
        for model_file in xp_yaml['model_files']:
            print('\n\n')
            print('running milp_model', milp_model)
            test_instance = MultiModelTaskTimesInstance( init_type='model_data_from_yaml',
                                             model_yaml=model_file, 
                                             sequence_length=SEQUENCE_LENGTH, 
                                             max_workers=NO_WORKERS, 
                                             no_stations=NO_STATIONS, 
                                             worker_cost=WORKER_COST, 
                                             recourse_cost=RECOURSE_COST)
            print('Running instance', test_instance.name)
            test_instances.append(test_instance)
            #create equipment
            if xp_yaml['equipment_files']:
               print('loading equipment from', xp_yaml['equipment_files'][0])
               equipment = Equipment(generation_method='import_yaml', equipment_file=xp_yaml['equipment_files'][0])
               if equipment.no_tasks != test_instance.no_tasks:
                  print('equipmen no tasks', equipment.no_tasks)
                  print('instance no tasks', test_instance.no_tasks)
                  #raises an error if the equipment and instance have different number of tasks
                  raise ValueError('Equipment and instance have different number of tasks')
            else:
               print('creating equipment')
               NO_EQUIPMENT = xp_yaml['no_equipment']
               equipment = Equipment(test_instance.no_tasks, 
                                     NO_EQUIPMENT, 
                                     NO_STATIONS, 
                                     generate_equipment, 
                                     seed)
            #create scenario tree
            print('generating scenario tree')
            model_mixtures = test_instance.model_mixtures
            scenario_tree_graph, final_sequences = scenario_generator(SEQUENCE_LENGTH, model_mixtures, **tree_kwargs)
            print('defining problem')
            milp_prob = milp_model(problem_instance = test_instance, 
                                   equipment_instance = equipment,  
                                   prod_sequences=final_sequences,
                                   env=env)
            start = timer()
            milp_prob.solve()
            end = timer()
            folder_name = None
            # if save_variables:
            #    folder_name = file_name + conf_name + str(group_counter) + test_instance.name+ '_variables/'
            #    if not os.path.exists(folder_name):
            #       os.makedirs(folder_name)
            #    milp_prob.save_variables(folder_name)
            # result = milp_prob.get_obj_val_dict()
            # result['equipment_instance'] = xp_yaml['equipment_files'][0]
            # result['xp_config_file'] = config_file
            # result['model_file'] = model_file
            # result['variables_folder'] = folder_name
            # result['run_time'] = end - start
            # result['group_counter'] = group_counter
            # result_df = pd.DataFrame([result], index=[0])
        #     if group_counter == 0:
        #        results_df = result_df.copy()
        #     else:
        #        results_df = pd.concat([results_df, result_df], axis=0, ignore_index=True)
        #     output_path=file_name + conf_name +  '_results.csv'
        #     results_df.to_csv(output_path)
        #     group_counter += 1
        # #deletes results df
        # del results_df
        end_time = time.time()
        print('time for', milp_model, end_time - start_time)
   return 1

In [3]:
run_from_config('SALBP_benchmark/MM_instances/small_instance_debug.yaml', base_file_name='model_runs')

Set parameter LogFile to value "gurobi.log"
Opening config file SALBP_benchmark/MM_instances/small_instance_debug.yaml
base_file_name model_runs
conf_name small_instance_debug
copying config file to results folder SALBP_benchmark/MM_instances/small_instance_debug.yaml model_runs/small_instance_debug_config.yaml



running milp_model <class 'gurobi_models.dynamic_MMALBP_W'>
using this file SALBP_benchmark/debugging_ds/instance_n=3_1.alb


TypeError: string indices must be integers